In [18]:
import tensorflow as tf
import numpy as np
import json
import h5py
import keras
from keras.models import Model,load_model,Sequential
from keras.layers import Dense,Conv2D,MaxPool2D,ReLU,Flatten,Permute
from keras.datasets import mnist,cifar10,fashion_mnist


In [3]:
model = Sequential()
model.add(Dense(units=24,activation='relu',input_shape=(784,)))
model.add(Dense(units=24,activation='relu'))
model.add(Dense(units=10,activation='softmax'))
model.compile(optimizer='Adam',loss='categorical_crossentropy',metrics=['accuracy'])
model.summary()


Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_4 (Dense)              (None, 24)                18840     
_________________________________________________________________
dense_5 (Dense)              (None, 24)                600       
_________________________________________________________________
dense_6 (Dense)              (None, 10)                250       
Total params: 19,690
Trainable params: 19,690
Non-trainable params: 0
_________________________________________________________________


In [4]:
for layer in model.layers:
    print(layer.name)

dense_4
dense_5
dense_6


In [5]:
model.save("test.h5")

In [3]:
f = h5py.File('eran_3_100.h5', 'r+')

for group in f.keys(): #可以查看所有的主键
    print(f[group].name)
    group_read = f[group]
    # 遍历该一级组下面的子组
    for subgroup in group_read.keys():
        print(subgroup)
        # 根据一级组和二级组名获取其下面的dataset
        dset_read = f[group + '/' + subgroup]
        # 遍历该子组下所有的dataset
        for dset in dset_read.keys():
            # 获取dataset数据
            dset1 = f[group + '/' + subgroup + '/' + dset]
            print('****   ',(group + '/' + subgroup + '/' + dset))
            print(dset1.name)
            data = np.array(dset1)
            print(data.shape)
            # weight = data[..., 0]
            # bias = data[..., 1]

/model_weights
dense_1
****    model_weights/dense_1/dense_1
/model_weights/dense_1/dense_1
(2,)
dense_2
****    model_weights/dense_2/dense_2
/model_weights/dense_2/dense_2
(2,)
dense_3
****    model_weights/dense_3/dense_3
/model_weights/dense_3/dense_3
(2,)


In [5]:
# import scipy.io as sio

dataFile = 'mnist24.mat'
# data = sio.loadmat(dataFile)
# data

import h5py
mat = h5py.File(dataFile)
# mat文件里可能有多个cell，各对应着一个dataset
# 可以用keys方法查看cell的名字
print(mat.keys())
# 可以用values方法查看各个cell的信息
# print(mat.values())
# 可以用shape查看维度信息
# print(mat['your_dataset_name'].shape)
# 注意，这里看到的shape信息与你在matlab打开的不同
# 这里的矩阵是matlab打开时矩阵的转置
# 所以，我们需要将它转置回来

mat_t = np.transpose(mat['fc1_bias'])
mat_t[0].shape


<KeysViewHDF5 ['fc1_bias', 'fc1_weight', 'fc2_bias', 'fc2_weight', 'fc3_bias', 'fc3_weight', 'net_struct']>


/home/ZhangZhaoDi/verifier/tensorflow2/env/lib/python3.7/site-packages/ipykernel_launcher.py:8: H5pyDeprecationWarning: The default file mode will change to 'r' (read-only) in h5py 3.0. To suppress this warning, pass the mode you need to h5py.File(), or set the global default h5.get_config().default_file_mode, or set the environment variable H5PY_DEFAULT_READONLY=1. Available modes are: 'r', 'r+', 'w', 'w-'/'x', 'a'. See the docs for details.
  


(24,)

In [21]:
dense_1_w = 'model_weights/dense_4/dense_4/kernel:0'
dense_1_b = 'model_weights/dense_4/dense_4/bias:0'
dense_2_w = 'model_weights/dense_5/dense_5/kernel:0'
dense_2_b = 'model_weights/dense_5/dense_5/bias:0'
dense_3_w = 'model_weights/dense_6/dense_6/kernel:0'
dense_3_b = 'model_weights/dense_6/dense_6/bias:0'

dtdense_1_w = np.transpose(mat['fc1_weight'])
dtdense_1_b = np.transpose(mat['fc1_bias'])[0]
dtdense_2_w = np.transpose(mat['fc2_weight'])
dtdense_2_b = np.transpose(mat['fc2_bias'])[0]
dtdense_3_w = np.transpose(mat['fc3_weight'])
dtdense_3_b = np.transpose(mat['fc3_bias'])[0]

del f[dense_1_w]
del f[dense_1_b]
del f[dense_2_w]
del f[dense_2_b]
del f[dense_3_w]
del f[dense_3_b]

f.create_dataset(dense_1_w, data=dtdense_1_w)
f.create_dataset(dense_1_b, data=dtdense_1_b)
f.create_dataset(dense_2_w, data=dtdense_2_w)
f.create_dataset(dense_2_b, data=dtdense_2_b)
f.create_dataset(dense_3_w, data=dtdense_3_w)
f.create_dataset(dense_3_b, data=dtdense_3_b)
f.close()


In [7]:
def preprocessing_data():
    (x_train, y_train), (x_test, y_test) = mnist.load_data()
    print(x_train.shape)
     # 归一化
    x_train = x_train / 255
    x_test = x_test / 255
    x_train = x_train.reshape(60000,784) # 将图片摊平，变成向量
    x_test = x_test.reshape(10000,784) # 对测试集进行同样的处理
    # 标签激活
    y_train = keras.utils.to_categorical(y_train, 10)
    y_test = keras.utils.to_categorical(y_test, 10)
    return x_train, y_train, x_test, y_test

In [7]:
x_train, y_train, x_test, y_test = preprocessing_data()
m = load_model("test.h5")
score = m.evaluate(x_test,y_test)
print("LOSS:",score[0],"ACCU:",score[1])

(60000, 28, 28)
313/313 [==============================] - 0s 454us/step - loss: 0.1316 - accuracy: 0.9660
LOSS: 0.13155309855937958 ACCU: 0.9660000205039978


In [30]:
import h5py
import scipy.io as sio
dataFile = 'eran_mnist_relu_6_100.mat'
# mat = h5py.File(dataFile)
# print(mat.keys())
mat = sio.loadmat(dataFile)
mat.keys()

dict_keys(['__header__', '__version__', '__globals__', 'fc1/weight', 'fc1/bias', 'fc2/weight', 'fc2/bias', 'fc3/weight', 'fc3/bias', 'fc4/weight', 'fc4/bias', 'fc5/weight', 'fc5/bias', 'fc6/weight', 'fc6/bias'])

In [20]:
mat['fc1/bias'][0].shape

(100,)

In [29]:
model = Sequential()
model.add(Dense(units=100,activation='relu',input_shape=(784,)))
model.add(Dense(units=100,activation='relu'))
model.add(Dense(units=100,activation='relu'))
model.add(Dense(units=100,activation='relu'))
model.add(Dense(units=100,activation='relu'))
model.add(Dense(units=10,activation='relu'))
model.compile(optimizer='Adam',loss='categorical_crossentropy',metrics=['accuracy'])
model.summary()
model.save("eran_6_100.h5")
for layer in model.layers:
    print(layer.name)

Model: "sequential_13"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_13 (Dense)             (None, 100)               78500     
_________________________________________________________________
dense_14 (Dense)             (None, 100)               10100     
_________________________________________________________________
dense_15 (Dense)             (None, 100)               10100     
_________________________________________________________________
dense_16 (Dense)             (None, 100)               10100     
_________________________________________________________________
dense_17 (Dense)             (None, 100)               10100     
_________________________________________________________________
dense_18 (Dense)             (None, 10)                1010      
Total params: 119,910
Trainable params: 119,910
Non-trainable params: 0
_______________________________________________

In [31]:
f = h5py.File('eran_6_100.h5', 'r+')
dd = ['model_weights/dense_13/dense_13/'
     ,'model_weights/dense_14/dense_14/'
     ,'model_weights/dense_15/dense_15/'
     ,'model_weights/dense_16/dense_16/'
     ,'model_weights/dense_17/dense_17/'
     ,'model_weights/dense_18/dense_18/']
for i,n in enumerate(dd):
    dense_w = n+'kernel:0'
    dense_b = n+'bias:0'
    dtdense_w = mat['fc'+str(i+1)+'/weight']
    dtdense_b = mat['fc'+str(i+1)+'/bias'][0]
    
    del f[dense_w]
    del f[dense_b]
    f.create_dataset(dense_w, data=dtdense_w)
    f.create_dataset(dense_b, data=dtdense_b)

f.close()

KeyError: 'fc0/weight'

In [8]:
x_train, y_train, x_test, y_test = preprocessing_data()
m = load_model("eran_3_100.h5")
score = m.evaluate(x_test,y_test)
print("LOSS:",score[0],"ACCU:",score[1])

(60000, 28, 28)
10000/10000 [==============================] - 0s 15us/step
LOSS: 0.5404417549610138 ACCU: 0.9654


In [26]:


input_shape = (28,28,1)
model = Sequential()
model.add(Conv2D(16, (4, 4), strides=(2, 2), padding="same", activation='relu', input_shape=input_shape))
model.add(Conv2D(32, (4, 4), strides=(2, 2), padding="same", activation='relu'))
model.add(Permute((3,1,2)))
model.add(Flatten())
model.add(Dense(units=100,activation='relu'))
model.add(Dense(units=10,activation='softmax'))
model.compile(optimizer='Adam',loss='categorical_crossentropy',metrics=['accuracy'])
model.summary()


Model: "sequential_12"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_23 (Conv2D)           (None, 14, 14, 16)        272       
_________________________________________________________________
conv2d_24 (Conv2D)           (None, 7, 7, 32)          8224      
_________________________________________________________________
permute_6 (Permute)          (None, 32, 7, 7)          0         
_________________________________________________________________
flatten_7 (Flatten)          (None, 1568)              0         
_________________________________________________________________
dense_11 (Dense)             (None, 100)               156900    
_________________________________________________________________
dense_12 (Dense)             (None, 10)                1010      
Total params: 166,406
Trainable params: 166,406
Non-trainable params: 0
_______________________________________________

In [27]:

def preprocessing_data2():
    (x_train, y_train), (x_test, y_test) = mnist.load_data()
    print(x_train.shape)
     # 归一化
    x_train = x_train / 255
    x_test = x_test / 255
    x_train = x_train.reshape(60000,28,28,1)
    x_test = x_test.reshape(10000,28,28,1)
    # 标签激活
    y_train = keras.utils.to_categorical(y_train, 10)
    y_test = keras.utils.to_categorical(y_test, 10)
    return x_train, y_train, x_test, y_test

x_train, y_train, x_test, y_test = preprocessing_data2()
model.fit(x_train,y_train,batch_size=128,epochs=2,validation_data=(x_test,y_test))

(60000, 28, 28)
Train on 60000 samples, validate on 10000 samples
Epoch 1/2
60000/60000 [==============================] - 3s 55us/step - loss: 0.3084 - accuracy: 0.9089 - val_loss: 0.1140 - val_accuracy: 0.9629
Epoch 2/2
60000/60000 [==============================] - 3s 49us/step - loss: 0.0898 - accuracy: 0.9732 - val_loss: 0.0655 - val_accuracy: 0.9798


In [28]:
model.save("check_convnet.h5")

In [23]:
model.layers[1].output_shape

(None, 7, 7, 32)

In [36]:
a=np.array([[1,2,3,4],[5,6,7,8]])

In [37]:
a.tolist()

[[1, 2, 3, 4], [5, 6, 7, 8]]